This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'xxxxxxxxxxxxxxxxxx'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-11-06&end_date=2019-11-06&api_key=" + API_KEY
req = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
req.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2019-11-06',
  'end_date': '2019-11-06',
  'frequency': 'daily',
  'data': [['2019-11-06',
    100.5,
    103.2,
    99.4,
    103.2,
    None,
    132904.0,
    13585029.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

## Assignment Tasks

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Assignment Solutions:

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
# Create the url for the year 2017 
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
afx_x_2017_json = requests.get(url).json()

#### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
# Retrive the column names from the json object (except the date as we will be using it for our dict label)
column_names = afx_x_2017_json['dataset_data']['column_names'][1:]
column_names

['Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
# Retrive the data from the json object
data = afx_x_2017_json['dataset_data']['data']

# Preview data from one day
data[0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [8]:
# Using dict comprehension to generate the data dict
afx_x_2017_dict = {day[0]: dict(zip(column_names, day[1:])) for day in data}

# Preview one value structure of the resulting dictionary
afx_x_2017_dict['2017-12-29']

{'Open': 51.76,
 'High': 51.94,
 'Low': 51.45,
 'Close': 51.76,
 'Change': None,
 'Traded Volume': 34640.0,
 'Turnover': 1792304.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
# The highest opening price for the time perios
max(day['Open'] for day in afx_x_2017_dict.values() if day['Open'] != None)

53.11

In [10]:
# The Lowest opening price for the time perios
min(day['Open'] for day in afx_x_2017_dict.values() if day['Open'] != None)

34.0

#### 4. What was the largest change in any one day (based on High and Low price)

In [11]:
max((day["High"] - day["Low"]) for day in afx_x_2017_dict.values() if day["High"] != None and day['Low'] != None)

2.8100000000000023

#### 5. What was the largest change between any two days (based on Closing Price)?

In [12]:
close_prices = [day['Close'] for day in afx_x_2017_dict.values() if day['Close'] != None]

# Calculate largest change between any two days
max(close_prices) - min(close_prices)

19.03

#### 6. What was the average daily trading volume during this year?

In [13]:
traded_volumes = [day['Traded Volume'] for day in afx_x_2017_dict.values() if day['Traded Volume'] != None]

# Calculate the total traded volume
total_traded_volume = 0
for traded_volume in traded_volumes:
    total_traded_volume = total_traded_volume + traded_volume
    
# Calculate average traded volume
total_traded_volume / len(traded_volumes)

89124.33725490196

#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [14]:
import statistics
 
statistics.median(sorted(traded_volumes))

76286.0